In [9]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.model_impl import WCMSLModel



In [14]:
model = WCMSLModel()
model.load(PROJECT_ROOT / "src" / "models" / "wcmsl_base.pkl")

In [11]:
while (a := input("Enter text (or 'exit' to quit): ")) not in ("exit", ''):
    prediction = model.predict(a)
    print(f"Prediction: {prediction}")

In [53]:
import pandas as pd

from tqdm import tqdm

#df1 = pd.read_csv(PROJECT_ROOT / "data" / "processed" / "balanced_length_filtered_test_dataset.csv")
df1 = pd.read_csv(PROJECT_ROOT / "data" / "processed" / "balanced_length_filtered_test_dataset.csv")

all = len(df1)
correct = 0

for _, row in tqdm(df1.iterrows(), total=all):
    text, ai = row['text'], row['generated']
    prediction = model.predict(text).result

    prediction_test = "AI" if ai == 1 else "HUMAN"
    
    if prediction_test == prediction:
        correct += 1

accuracy = correct / all
print(f"Accuracy on AI Generated Essays Dataset: {accuracy:.4f}")

 20%|██        | 601/3000 [00:03<00:11, 200.04it/s]


KeyboardInterrupt: 